In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from My_Loss import HardTripletLoss2
from My_Loss import HardTripletLoss_D
from tensorboardX import SummaryWriter
import visdom

In [2]:
BATCH_SIZE = 64
EPISODE = 100000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Weight_Deacy = 0
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)

attribute_network.cuda(GPU) 
metric_network.cuda(GPU)

#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=20000 , gamma=0.5)
metric_network_scheduler = StepLR(metric_network_optim , step_size=20000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    

    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
   
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [ ]:
print("training...")
last_accuracy = 0.0
loss_zero_num =0
#writer = SummaryWriter()
#vis = visdom.Visdom()
for episode in range(EPISODE):

    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)


    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss2(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    
    
    attribute_network.zero_grad()
    metric_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()

    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        #H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#features = images.view(, 784)
#print(vis_label.size())
#writer.add_embedding(vis_images, metadata = vis_label.tolist())
#writer.add_embedding(vis_att, metadata = vis_label_att)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.0932, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.12162425891607524
gzsl: unseen=0.0000 , seen=0.0245 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(2.9842, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4264552223507801
gzsl: unseen=0.0245 , seen=0.1303 , h=0.0413
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9290, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.403873122957714
gzsl: unseen=0.0186 , seen=0.1516 , h=0.0332
____________________________________________________________________________________________________
episode: 600 loss tensor(2.9142, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 5

episode: 5600 loss tensor(1.9128, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6538143415634057
gzsl: unseen=0.5271 , seen=0.7122 , h=0.6058
____________________________________________________________________________________________________
episode: 5800 loss tensor(1.3677, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6481596888012753
gzsl: unseen=0.5203 , seen=0.7103 , h=0.6006
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.7895, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6575590424229515
gzsl: unseen=0.5288 , seen=0.7195 , h=0.6096
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.8735, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
clas

episode: 11200 loss tensor(1.7958, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6762303508647435
gzsl: unseen=0.5423 , seen=0.7595 , h=0.6327
____________________________________________________________________________________________________
episode: 11400 loss tensor(1.0485, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6836909355516853
gzsl: unseen=0.5526 , seen=0.7578 , h=0.6391
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.9172, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6780485153051883
gzsl: unseen=0.5393 , seen=0.7536 , h=0.6287
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.2086, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


episode: 16800 loss tensor(1.7022, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6582300762163981
gzsl: unseen=0.5083 , seen=0.7725 , h=0.6131
____________________________________________________________________________________________________
episode: 17000 loss tensor(2.9063, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.653396677230081
gzsl: unseen=0.5225 , seen=0.7644 , h=0.6207
____________________________________________________________________________________________________
episode: 17200 loss tensor(1.6712, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6546823248016352
gzsl: unseen=0.5125 , seen=0.7769 , h=0.6176
____________________________________________________________________________________________________
episode: 17400 loss tensor(2.2994, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
c

episode: 22400 loss tensor(1.7417, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6560806570596208
gzsl: unseen=0.5039 , seen=0.7779 , h=0.6116
____________________________________________________________________________________________________
episode: 22600 loss tensor(1.0504, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6467961377729365
gzsl: unseen=0.5045 , seen=0.7801 , h=0.6128
____________________________________________________________________________________________________
episode: 22800 loss tensor(1.6452, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6439103001677999
gzsl: unseen=0.4963 , seen=0.7816 , h=0.6071
____________________________________________________________________________________________________
episode: 23000 loss tensor(1.8151, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50


...

episode: 56000 loss tensor(0.7228, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6252028791819744
gzsl: unseen=0.4746 , seen=0.7935 , h=0.5940
____________________________________________________________________________________________________
episode: 56200 loss tensor(1.2920, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6265484483672272
gzsl: unseen=0.4716 , seen=0.7922 , h=0.5912
____________________________________________________________________________________________________
episode: 56400 loss tensor(1.4223, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6328830624724192
gzsl: unseen=0.4727 , seen=0.7962 , h=0.5932
____________________________________________________________________________________________________
episode: 56600 loss tensor(0.1490, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50


episode: 61600 loss tensor(1.2296, device='cuda:0')
loss_zero_number=  3
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6261219543259424
gzsl: unseen=0.4807 , seen=0.7890 , h=0.5975
____________________________________________________________________________________________________
episode: 61800 loss tensor(0.5193, device='cuda:0')
loss_zero_number=  4
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6265365111129144
gzsl: unseen=0.4758 , seen=0.7934 , h=0.5948
____________________________________________________________________________________________________
episode: 62000 loss tensor(2.4915, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6260019548113005
gzsl: unseen=0.4784 , seen=0.7903 , h=0.5960
____________________________________________________________________________________________________
episode: 62200 loss tensor(1.7737, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50


episode: 67200 loss tensor(1.1336, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6226151008540127
gzsl: unseen=0.4773 , seen=0.7959 , h=0.5967
____________________________________________________________________________________________________
episode: 67400 loss tensor(1.4853, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6203260355123221
gzsl: unseen=0.4756 , seen=0.7922 , h=0.5944
____________________________________________________________________________________________________
episode: 67600 loss tensor(1.4368, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.622115304629551
gzsl: unseen=0.4774 , seen=0.7916 , h=0.5956
____________________________________________________________________________________________________
episode: 67800 loss tensor(0.7718, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
c

episode: 72800 loss tensor(2.0477, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6289376731825099
gzsl: unseen=0.4765 , seen=0.7978 , h=0.5966
____________________________________________________________________________________________________
episode: 73000 loss tensor(0.9898, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6270140394555972
gzsl: unseen=0.4743 , seen=0.7928 , h=0.5935
____________________________________________________________________________________________________
episode: 73200 loss tensor(0.8999, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6300261983292618
gzsl: unseen=0.4740 , seen=0.7982 , h=0.5948
____________________________________________________________________________________________________
episode: 73400 loss tensor(0.9914, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50


episode: 78400 loss tensor(1.5708, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6220442749133093
gzsl: unseen=0.4736 , seen=0.7919 , h=0.5927
____________________________________________________________________________________________________
episode: 78600 loss tensor(1.9030, device='cuda:0')
loss_zero_number=  2
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6192026148824982
gzsl: unseen=0.4658 , seen=0.7943 , h=0.5873
____________________________________________________________________________________________________
episode: 78800 loss tensor(1.9176, device='cuda:0')
loss_zero_number=  1
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6204953795342876
gzsl: unseen=0.4647 , seen=0.7949 , h=0.5866
____________________________________________________________________________________________________
episode: 79000 loss tensor(1.3742, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 10
class num: 50
